# Test BigData

In [ ]:
# jupyter nbconvert --to script BigData.ipynb

import os
import time
# import azure.cosmos
# import azure.cosmos.cosmos_client as cosmos_client
from runpy import run_path
from azure.cosmos import exceptions, CosmosClient, PartitionKey, documents # https://github.com/Azure-Samples/azure-cosmos-db-python-getting-started/blob/master/cosmos_get_started.py

credentials = run_path("/home/opc/python/cosmos-notebooks/credentials.py")

endpoint = credentials['HOST']
key = credentials['MASTER_KEY']

def isnotebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

In [ ]:
'''
# https://github.com/MicrosoftDocs/azure-docs/blob/master/articles/cosmos-db/tutorial-global-distribution-sql-api.md
connectionPolicy = documents.ConnectionPolicy()
connectionPolicy.PreferredLocations = [ 'North Europe', 'Southeast Asia', 'East US', 'East US 2']
{"code":"BadRequest","message":"Value '<azure.cosmos.documents.ConnectionPolicy object at 0x7f8eab687b00>' specified for the header 'x-ms-consistency-level' is invalid. \r\nActivityId: e1c30f0c-6dac-4885-83a5-b01afb5853e6, Microsoft.Azure.Documents.Common/2.7.0"}
'''
# <create_cosmos_client>
client = CosmosClient(endpoint, {'masterKey': key} )
# </create_cosmos_client>

# Create a database
# <create_database_if_not_exists>
database_name = 'mybigdata'
database = client.create_database_if_not_exists(id=database_name)
# </create_database_if_not_exists>

In [ ]:
# Create a container
# Using a good partition key improves the performance of database operations.
# <create_container_if_not_exists>
container_name = 'myitems'
container = database.create_container_if_not_exists(
    id=container_name, 
    partition_key=PartitionKey(path="/id"),
    offer_throughput=5000
)
# </create_container_if_not_exists>

## Store random data

In [ ]:
import time
import uuid

progress = 1
max_count = 20000000
for i in range(max_count):
    new_item = { 'id': str(uuid.uuid4()), 'timestamp': time.time(), 'dummy': str(uuid.uuid4()) }
    container.create_item(new_item)
    if progress % 10 == 0:
        print("Inserting document: %5d / \r"% (progress), end="", flush=True)
    progress += 1

## Read and process data line by line

In [ ]:

# jupyter nbconvert --to script BigData.ipynb
# Read a CSV file in Python, libe-by-line, by Jeff Heaton (http://www.jeffheaton.com/tutorials/)
import codecs
import csv

FILENAME = "/home/cosmosuser/notebooks/census/Rate.csv" # https://www.kaggle.com/hhs/health-insurance-marketplace#Rate.csv
ENCODING = 'utf-8'

with codecs.open(FILENAME, "r", ENCODING) as fp:
    reader = csv.reader(fp)

    # read CSV headers
    headers = next(reader)
    print(headers)

    # read rest of file
    for row in reader:
        # Print each row
        print(row)

        # Print individual fields of the row
        # print("{},{},{},{} = {}".format(row[0],row[1],row[2],row[3],row[4]))

In [ ]:
if isnotebook():
    client.delete_database(database_name)